In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['DC', 'DCs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('dendritic cells', 18763),
 ('dyskeratosis congenita', 60),
 ('direct current', 57),
 ('decompressive craniectomy', 54),
 ('deoxycholate', 33),
 ('dc', 20),
 ('doxycycline', 17),
 ('dorsal column', 14),
 ('dorsal closure', 13),
 ('deceleration capacity', 12),
 ('dark chocolate', 11),
 ('decisional conflict', 10),
 ('degree central', 9),
 ('diabetic cardiomyopathy', 8),
 ('dentritic cells', 8),
 ('deoxycholate acid', 7),
 ('dabigatran concentration', 6),
 ('dopachrome', 6),
 ('dansylcadaverine', 6),
 ('differentiation cells', 6),
 ('diene conjugates', 5),
 ('dosage compensation', 5),
 ('direct electric current', 5),
 ('dendritic antigen presenting cells', 5),
 ('destruction complex', 4),
 ('dotap cholesterol', 4),
 ('decylcatechol', 4),
 ('diclofenac', 4),
 ('dupuytren s contracture', 4),
 ('degree of vinyl conversion', 4),
 ('drug content', 3),
 ('decompressive craniotomy', 3),
 ('dental caries', 3),
 ('distensibility coefficient', 3),
 ('dietary cholesterol', 3),
 ('decanoylcarnitin

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'MESH:D003713': 'Dendritic Cells',
 'MESH:D019871': 'Dyskeratosis Congenita',
 'MESH:D056424': 'Decompressive Craniectomy',
 'CHEBI:CHEBI:23614': 'deoxycholate',
 'HGNC:2714': 'DCX',
 'CHEBI:CHEBI:50845': 'doxycycline',
 'GO:GO:0007391': 'dorsal closure',
 'MESH:D000069956': 'Chocolate',
 'HP:HP:0030645': 'Central',
 'MESH:D058065': 'Diabetic Cardiomyopathies',
 'MESH:D002477': 'Cells',
 'CHEBI:CHEBI:49108': 'dopachrome',
 'CHEBI:CHEBI:52007': 'monodansylcadaverine',
 'MESH:D000938': 'Antigen-Presenting Cells',
 'CHEBI:CHEBI:16113': 'cholesterol',
 'CHEBI:CHEBI:47381': 'diclofenac',
 'MESH:D004387': 'Dupuytren Contracture',
 'MESH:D003399': 'Craniotomy',
 'MESH:D003731': 'Dental Caries',
 'MESH:D002791': 'Cholesterol, Dietary',
 'MESH:C002893': 'decanoylcarnitine',
 'CHEBI:CHEBI:4513': 'dicoumarol',
 'CHEBI:CHEBI:36281': 'caffeic acid',
 'MESH:D003654': 'Deceleration',
 'HGNC:2705': 'DCN',
 'MESH:C101278': 'decursin',
 'CHEBI:CHEBI:4354': 'decursinol',
 'PUBCHEM:13848091': '3-Decylben

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-03 02:39:40] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'dabigatran concentration': 'ungrounded',
  'dansylcadaverine': 'CHEBI:CHEBI:52007',
  'dark chocolate': 'ungrounded',
  'dc': 'ungrounded',
  'decanoylcarnitine': 'MESH:C002893',
  'deceleration capacity': 'ungrounded',
  'decisional conflict': 'ungrounded',
  'decompressive craniectomy': 'MESH:D056424',
  'decompressive craniotomy': 'ungrounded',
  'decyl caffeic acid': 'CHEBI:CHEBI:36281',
  'decylcatechol': 'PUBCHEM:13848091',
  'degree central': 'ungrounded',
  'degree of vinyl conversion': 'ungrounded',
  'dehydrocavidine': 'ungrounded',
  'deiters s cells': 'ungrounded',
  'dendritic antigen presenting cells': 'MESH:D000938',
  'dendritic cells': 'MESH:D003713',
  'dental caries': 'MESH:D003731',
  'dentritic cells': 'MESH:D003713',
  'deoxycholate': 'CHEBI:CHEBI:23614',
  'deoxycholate acid': 'ungrounded',
  'destruction complex': 'MESH:D060465',
  'diabetic cardiomyopathy': 'MESH:D058065',
  'diclofenac': 'CHEBI:CHEBI:47381',
  'dicoumarol': 'CHEBI:CHEBI:4513',
  'diene conj

In [23]:
grounding_map, names, pos_labels = [{'dabigatran concentration': 'ungrounded',
  'dansylcadaverine': 'CHEBI:CHEBI:52007',
  'dark chocolate': 'ungrounded',
  'dc': 'ungrounded',
  'decanoylcarnitine': 'MESH:C002893',
  'deceleration capacity': 'ungrounded',
  'decisional conflict': 'ungrounded',
  'decompressive craniectomy': 'MESH:D056424',
  'decompressive craniotomy': 'ungrounded',
  'decyl caffeic acid': 'CHEBI:CHEBI:36281',
  'decylcatechol': 'PUBCHEM:13848091',
  'degree central': 'ungrounded',
  'degree of vinyl conversion': 'ungrounded',
  'dehydrocavidine': 'ungrounded',
  'deiters s cells': 'ungrounded',
  'dendritic antigen presenting cells': 'MESH:D000938',
  'dendritic cells': 'MESH:D003713',
  'dental caries': 'MESH:D003731',
  'dentritic cells': 'MESH:D003713',
  'deoxycholate': 'CHEBI:CHEBI:23614',
  'deoxycholate acid': 'ungrounded',
  'destruction complex': 'MESH:D060465',
  'diabetic cardiomyopathy': 'MESH:D058065',
  'diclofenac': 'CHEBI:CHEBI:47381',
  'dicoumarol': 'CHEBI:CHEBI:4513',
  'diene conjugates': 'ungrounded',
  'dietary cholesterol': 'MESH:D002791',
  'differentiation cells': 'ungrounded',
  'direct current': 'ungrounded',
  'direct electric current': 'ungrounded',
  'distensibility coefficient': 'ungrounded',
  'dopachrome': 'CHEBI:CHEBI:49108',
  'dorsal closure': 'GO:GO:0007391',
  'dorsal column': 'ungrounded',
  'dosage compensation': 'GO:GO:0007549',
  'dotap cholesterol': 'ungrounded',
  'doxycycline': 'CHEBI:CHEBI:50845',
  'drug content': 'ungrounded',
  'dupuytren s contracture': 'ungrounded',
  'dyskeratosis congenita': 'MESH:D019871'},
 {'CHEBI:CHEBI:52007': 'monodansylcadaverine',
  'MESH:C002893': 'decanoylcarnitine',
  'MESH:D056424': 'Decompressive Craniectomy',
  'CHEBI:CHEBI:36281': 'caffeic acid',
  'PUBCHEM:13848091': '3-Decylbenzene-1,2-diol',
  'MESH:D000938': 'Antigen-Presenting Cells',
  'MESH:D003713': 'Dendritic Cells',
  'MESH:D003731': 'Dental Caries',
  'CHEBI:CHEBI:23614': 'deoxycholate',
  'MESH:D060465': 'Axin Signaling Complex',
  'MESH:D058065': 'Diabetic Cardiomyopathies',
  'CHEBI:CHEBI:47381': 'diclofenac',
  'CHEBI:CHEBI:4513': 'dicoumarol',
  'MESH:D002791': 'Cholesterol, Dietary',
  'CHEBI:CHEBI:49108': 'dopachrome',
  'GO:GO:0007391': 'dorsal closure',
  'GO:GO:0007549': 'dosage compensation',
  'CHEBI:CHEBI:50845': 'doxycycline',
  'MESH:D019871': 'Dyskeratosis Congenita'},
 ['CHEBI:CHEBI:23614',
  'CHEBI:CHEBI:50845',
  'MESH:D003713',
  'MESH:D019871',
  'MESH:D056424']]

In [24]:
excluded_longforms = ['dc']

In [25]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [26]:
additional_entities = {'HGNC:2714': ['DCX', ['DC']]}

In [27]:
unambiguous_agent_texts = {''

In [28]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [29]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [30]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [33]:
intersection1

[('HGNC:2714', 'HGNC:2714', 0)]

In [34]:
intersection2

[('CHEBI:CHEBI:23614', 'HGNC:2714', 0),
 ('ungrounded', 'HGNC:2714', 0),
 ('MESH:D003713', 'HGNC:2714', 0),
 ('CHEBI:CHEBI:52007', 'HGNC:2714', 0),
 ('CHEBI:CHEBI:49108', 'HGNC:2714', 0),
 ('CHEBI:CHEBI:47381', 'HGNC:2714', 0),
 ('MESH:D000938', 'HGNC:2714', 0),
 ('MESH:C002893', 'HGNC:2714', 0),
 ('CHEBI:CHEBI:50845', 'HGNC:2714', 0),
 ('GO:GO:0007391', 'HGNC:2714', 0),
 ('MESH:D056424', 'HGNC:2714', 0),
 ('MESH:D019871', 'HGNC:2714', 0),
 ('MESH:D002791', 'HGNC:2714', 0),
 ('CHEBI:CHEBI:4513', 'HGNC:2714', 0),
 ('MESH:D058065', 'HGNC:2714', 0),
 ('MESH:D060465', 'HGNC:2714', 0),
 ('GO:GO:0007549', 'HGNC:2714', 0),
 ('MESH:D003731', 'HGNC:2714', 0),
 ('PUBCHEM:13848091', 'HGNC:2714', 0),
 ('CHEBI:CHEBI:36281', 'HGNC:2714', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-03 02:52:30] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-03 03:01:09] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.995340248308574 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:23614': 19,
  'ungrounded': 100,
  'MESH:D003713': 15989,
  'CHEBI:CHEBI:52007': 5,
  'CHEBI:CHEBI:49108': 5,
  'CHEBI:CHEBI:47381': 3,
  'MESH:D000938': 5,
  'MESH:C002893': 3,
  'CHEBI:CHEBI:50845': 10,
  'GO:GO:0007391': 5,
  'MESH:D056424': 39,
  'MESH:D019871': 37,
  'MESH:D002791': 2,
  'CHEBI:CHEBI:4513': 3,
  'MESH:D058065': 6,
  'MESH:D060465': 4,
  'GO:GO:0007549': 4,
  'MESH:D003731': 2,
  'PUBCHEM:13848091': 2,
  'CHEBI:CHEBI:36281': 1},
 'f1': {'mean': 0.99534, 'std': 0.000457},
 'precision': {'mean': 0.993137, 'std': 0.000491},
 'recall': {'mean': 0.997701, 'std': 0.000542},
 'CHEBI:CHEBI:49108': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'CHEBI:CHEBI:23614': {'f1': {'mean': 0.533333, 'std': 0.193781},
  'pr': {'mean': 0.466667, 'std': 0.171594},
  'rc': {'mean': 0.633333, 'std': 0.221108}},
 'MESH:D056424': {'f1': {'mean': 0.862081, 'std': 0.124228},
  'pr': {'mean': 0.814

In [19]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [20]:
disamb.dump(model_name, results_path)

In [21]:
print(disamb.info())

Disambiguation model for DC, and DCs

Produces the disambiguations:
	3-Decylbenzene-1,2-diol	PUBCHEM:13848091
	Antigen-Presenting Cells	MESH:D000938
	Axin Signaling Complex	MESH:D060465
	Cholesterol, Dietary	MESH:D002791
	Decompressive Craniectomy*	MESH:D056424
	Dendritic Cells*	MESH:D003713
	Dental Caries	MESH:D003731
	Diabetic Cardiomyopathies	MESH:D058065
	Dyskeratosis Congenita*	MESH:D019871
	caffeic acid	CHEBI:CHEBI:36281
	decanoylcarnitine	MESH:C002893
	deoxycholate*	CHEBI:CHEBI:23614
	diclofenac	CHEBI:CHEBI:47381
	dicoumarol	CHEBI:CHEBI:4513
	dopachrome	CHEBI:CHEBI:49108
	dorsal closure	GO:GO:0007391
	dosage compensation	GO:GO:0007549
	doxycycline*	CHEBI:CHEBI:50845
	monodansylcadaverine	CHEBI:CHEBI:52007

Class level metrics:
--------------------
Grounding                	Count	F1     
          Dendritic Cells*	15989	0.99698
               Ungrounded	  100	0.69485
Decompressive Craniectomy*	   39	0.86208
   Dyskeratosis Congenita*	   37	0.92256
             deoxycholate*	   19

In [22]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1